<a href="https://colab.research.google.com/github/fcolombo7/AN2DL-2020/blob/main/Final%20Notebooks/5_K_Fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AN2DL** - Image classification challenge

* **Colombo** Filippo - 10559531
* **Del Vecchio** Giovanni - 10570682


## K-Fold cross validation 


Configuration and constants:

In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
#Random seed to make experiments reproducible
SEED = 1234
tf.random.set_seed(SEED)  

#Parameters
IMG_H, IMG_W = (448, 448)
BS = 32 #BATCH SIZE
VALIDATION_SPLIT = 0.2
DATA_AUGMENTATION = True

Load Google Drive to get the data and save the results:

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

cwd = os.getcwd()
drive_root_folder = '/content/drive/My Drive/ANN_project/'

Mounted at /content/drive


### Import the Mask Dataset

Check if the dataset has been already processed, otherwise unzip it.

**N.B.**: we are unzipping in the `/content` folder and not in `drive`.  
If you want to unzip in your drive folder, write `os.chdir(drive_root_folder)`



In [ ]:
#check if the dataset is already available
if not os.path.exists(cwd+'/MaskDataset'):
  !unzip '/content/drive/My Drive/ANN_project/artificial-neural-networks-and-deep-learning-2020.zip'
else:
  print('MaskDataset already loaded')

#### Define the generator

Definition of the `ImageDataGenerator` for _data augmentation_:

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

if DATA_AUGMENTATION:
    train_data_gen = ImageDataGenerator(rotation_range=20, 
                                        width_shift_range=0.3, 
                                        height_shift_range=0.3, 
                                        zoom_range=0.4, 
                                        horizontal_flip=True, 
                                        shear_range=10, 
                                        channel_shift_range=100, 
                                        fill_mode='reflect', 
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)
    
valid_data_gen = ImageDataGenerator(rescale=1./255)

### K-Fold procedure

In [ ]:
def get_optimizer(lr_schedule = 1e-3):
  return tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [ ]:
def get_callbacks(name, es=True, LR = False , patience = 10):
  obj = []

  exps_dir = os.path.join(drive_root_folder,'classification_result')
  if not os.path.exists(exps_dir):
    os.mkdir(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')
  temp_dir = name

  #Model Checkpoints
  ckpt_dir = os.path.join(exps_dir, temp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

  obj.append(tf.keras.callbacks.ModelCheckpoint(filepath = os.path.join(ckpt_dir, 'cp.ckpt'),
                                                save_weights_only = True,
                                                save_best_only = True))
  
  #Tensor board logs
  tb_dir = os.path.join(exps_dir, temp_dir + '/tb_logs')
  if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
  obj.append(tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                            profile_batch = 0,
                                            histogram_freq = 1))
  if es:
    obj.append(tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = patience))

  if LR:
    obj.append(tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    mode="auto",
    min_lr=0.0001
    ))

  return obj

In [ ]:
from sklearn.model_selection import KFold

np.random.seed(SEED)

import json
#from sklearn.model_selection import train_test_split

dataset_dir = os.path.join(cwd, "MaskDataset")
training_dir = os.path.join(dataset_dir, "training")
with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic = json.load(f)
  dataframe = pd.DataFrame(dic.items())
  dataframe.rename(columns = {0:'filename', 1:'class'}, inplace = True)
  dataframe = dataframe.sample(frac=1, random_state=SEED)

In [ ]:
#auxiliary variable to handle Colab's error due to GPU's exhaustion or random runtime disconnections
stopped_train = False
stopped_train_ft = False

kf = KFold(n_splits = 5, shuffle = True, random_state = SEED)
model_number = 0
skip_first = -1
for result in kf.split(dataframe):
  if model_number > skip_first:
    train = dataframe.iloc[result[0]]
    valid = dataframe.iloc[result[1]]
    train["class"] = train["class"].astype('string')
    valid["class"] = valid["class"].astype('string')
    
    train_gen = train_data_gen.flow_from_dataframe(train,
                                                training_dir,
                                                batch_size=BS,
                                                target_size=(IMG_H, IMG_W),
                                                class_mode='categorical',
                                                shuffle=True,
                                                seed=SEED)
    
    validation_gen = valid_data_gen.flow_from_dataframe(valid,
                                                training_dir,
                                                batch_size=BS,
                                                target_size=(IMG_H, IMG_W),
                                                class_mode='categorical',
                                                shuffle=True,
                                                seed=SEED)
    #create the dataset objects
    train_ds = tf.data.Dataset.from_generator(lambda: train_gen, 
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes= ([None, IMG_H, IMG_W, 3], [None, 3]))

    train_ds = train_ds.repeat()

    valid_ds = tf.data.Dataset.from_generator(lambda: validation_gen,
                                              output_types = (tf.float32, tf.float32),
                                              output_shapes = ([None, IMG_H, IMG_W, 3], [None, 3]))
    valid_ds = valid_ds.repeat()

    name = 'VGG_512+448_' + str(model_number)

    base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(IMG_H,IMG_W,3))
    base_model.trainable = False
        
    model = tf.keras.Sequential()
    model.add(base_model)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=512, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(units=448, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(units=3, activation='softmax'))
    model.compile(optimizer=get_optimizer(lr_schedule=1e-4),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])
    
    if stopped_train and model_number == (skip_first+1):
      model_to_load = 'VGG_512+448_' + str(model_number)
      ckpt_path = os.path.join(drive_root_folder,'classification_result',model_to_load, 'ckpts','cp.ckpt')
      model.load_weights(ckpt_path)

    print('\nTrain' + str(model_number) + '\n')
    model.fit(x=train_ds,
              epochs=30, 
              steps_per_epoch = len(train_gen),
              validation_data = valid_ds,
              validation_steps = len(validation_gen), 
              callbacks = get_callbacks(name),
              verbose = 1)
    
    model_to_load = 'VGG_512+448_' + str(model_number)
    ckpt_path = os.path.join(drive_root_folder,'classification_result',model_to_load, 'ckpts','cp.ckpt')
    model.load_weights(ckpt_path)
    base_model.trainable = True

    if stopped_train_ft and model_number == (skip_first+1):
      model_to_load = 'FT-VGG_512+448_' + str(model_number)
      ckpt_path = os.path.join(drive_root_folder,'classification_result',model_to_load, 'ckpts','cp.ckpt')
      model.load_weights(ckpt_path)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),  # Low learning rate
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )
    print('\nFine Tuning\n')
    epochs = 60
    model.fit(train_ds,
              epochs = epochs,
              validation_data = valid_ds,
              steps_per_epoch = len(train_gen),
              validation_steps = len(validation_gen),
              callbacks=get_callbacks('FT-VGG_512+448_' + str(model_number), es = True, patience=12))
    
  model_number += 1

## Predictions with majority voting

First load all the k models trained before: 

In [ ]:
def remodel():
  base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(IMG_H,IMG_W,3))
  base_model.trainable = False
      
  model = tf.keras.Sequential()
  model.add(base_model)
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
  model.add(tf.keras.layers.Dropout(0.1))
  model.add(tf.keras.layers.Dense(units=3, activation='softmax'))
  model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
  return model

In [ ]:
def load_all_models(n_models=5):
  all_models = list()
  for n in range(n_models):
    model = remodel()
    model_to_load = 'FT-VGG16_' + str(n)
    ckpt_path = os.path.join(drive_root_folder,'classification_result',model_to_load, 'ckpts','cp.ckpt')
    model.load_weights(ckpt_path)
    all_models.append(model)
    print('>loaded %s' %ckpt_path)
  return all_models

Predictions are performed taking care of all the output probabilities of the k models:

In [ ]:
def ensemble_predictions(members, testX):
  # make predictions
  yhats = [model.predict(testX) for model in members]
  yhats = np.array(yhats)
  # sum across ensemble members
  summed = np.sum(yhats, axis=0)
  # argmax across classes
  result = np.argmax(summed, axis=1)
  return result

Then load the test images:

In [ ]:
import pathlib
path = os.path.join(os.getcwd(),'MaskDataset', 'test')
data_dir = pathlib.Path(path)

test_image_filenames = list(data_dir.glob('*.jpg'))

In [ ]:
test_normalization = tf.keras.Sequential([
                                       tf.keras.layers.experimental.preprocessing.Rescaling(1./255)                 
])

For each image the output class is the best accordin to all the five models:

In [ ]:
def make_prediction(all_models):
  results = {}
  results_expanded = {}
  for test_image in test_image_filenames:
    img = tf.keras.preprocessing.image.load_img(test_image, target_size=(IMG_H, IMG_W))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    normalized_img = test_normalization(img_array, training=True)

    result = ensemble_predictions(all_models, normalized_img)
    results_expanded[os.path.basename(test_image)]=result
    results[os.path.basename(test_image)]=result[0]
  return results, results_expanded

In [ ]:
def create_csv(results, results_dir):
    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:
        f.write('Id,Category\n')
        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
create_csv(make_prediction(load_all_models())[0], drive_root_folder)